In [1]:
import numpy as np
import pandas as pd

In [2]:
input_fname = '../../Train_Tagged_Titles/Train_Tagged_Titles.tsv'

In [3]:
df = pd.read_csv(input_fname, sep='\t')

In [4]:
df.drop(['Unnamed: 4'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
# df.head(30)
print(df.iloc[0])

Record Number                                                    1
Title            Supreme Nike SB Dunk High By any Means Red US1...
Token                                                      Supreme
Tag                                                         Modell
Name: 0, dtype: object


In [6]:
print(len(df))

52005


In [7]:
lim = len(df)
i = 0

while i < lim:    
    row = df.iloc[i]
    
    if row['Tag'] is np.NaN:
        df.iloc[i-1]['Token'] = str(df.iloc[i-1]['Token']) + ' ' + str(row['Token'])
        df.drop([i], inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        lim = len(df)
    else:
        i += 1

In [ ]:
# df.head(30)

# Language Detector

In [8]:
from spacy_langdetect import LanguageDetector
import spacy

2023-05-25 22:02:23.186820: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 22:02:23.452125: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-25 22:02:23.452225: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-25 22:02:24.589963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [9]:
# df.info()

In [10]:
from spacy.language import Language

In [11]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlpLD = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlpLD.add_pipe('language_detector', last=True)

In [12]:
text = 'Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force'
text2 = 'NEU SKECHERS Damen Sneakers Turnschuhe Memory Foam MODA - LAZY SUNDAYS Blau'
doc = nlpLD(text)
print(doc._.language)

{'language': 'en', 'score': 0.9999965162629252}


In [35]:
def detect_language(text):
    doc = nlpLD(text)
    det = doc._.language
        
    return det['language']

In [33]:
detect_language('MICHAEL KORS Sneaker Gr 39')

{'language': 'no', 'score': 0.9999972714104637}


'no'

# spaCy NER

In [46]:
import spacy
from spacy.tokens import DocBin
import re
import emoji

regex_specials = ["\\", ".", "+", "*", "?", "^", "$", "(", ")", "[", "]", "{", "}", "|"]

In [47]:
nlpEN = spacy.blank('en')
nlpDE = spacy.blank('de')
db = DocBin()

In [49]:
for idx, row in df.iterrows():
    text = row['Title']
    token = row['Token']
    
    text = re.sub(r':[^\s]*:', '', emoji.demojize(text))
    
    for reg_special in regex_specials:
        token = token.replace(reg_special, '\\'+reg_special)
    print(text + '\t\t' + token)
    
    doc = None
    if detect_language(text) == 'de':
        doc = nlpDE.make_doc(text)
    else:
        doc = nlpEN.make_doc(text)
        
    ents = []
    for match in re.finditer(token, text):
        span = doc.char_span(match.span()[0], match.span()[1], label = row['Tag'], alignment_mode='contract')
        if span is None:
            print('Skipping entity')
        else:
            ents.append(span)
    
    doc.ents = ents
    db.add(doc)